In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import rasterio
import xarray as xr

In [7]:
# Open the GeoTIFF
with rasterio.open("Maharashtra_O3_July2024-Dec2024.tif") as src:
    data = src.read(1)  # first band
    coords = {
        "y": src.xy(0, 0)[1] - src.res[1] * np.arange(src.height),  # latitude
        "x": src.xy(0, 0)[0] + src.res[0] * np.arange(src.width)   # longitude
    }
    o3_da = xr.DataArray(data, dims=("y","x"), coords=coords)

print(o3_da)

<xarray.DataArray (y: 103, x: 132)> Size: 109kB
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * y        (y) float64 824B 22.04 21.98 21.91 21.85 ... 15.75 15.69 15.63
  * x        (x) float64 1kB 72.66 72.72 72.79 72.85 ... 80.71 80.77 80.83 80.9


In [15]:
df = pd.DataFrame({
    "longitude": np.repeat(o3_da.x.values, o3_da.y.size),
    "latitude": np.tile(o3_da.y.values, o3_da.x.size),
    "O3": o3_da.values.flatten()
})
df = df.dropna().reset_index(drop=True)
df.head(100)

,longitude,latitude,O3
0,72.660232,20.342350,0.123790
1,72.660232,20.279468,0.123900
2,72.723114,18.770298,0.123742
3,72.723114,18.707416,0.123740
4,72.723114,18.581652,0.123744
...,...,...,...
95,73.163288,19.210472,0.123050
96,73.163288,19.147590,0.122966
97,73.163288,17.575539,0.123183
98,73.163288,17.512656,0.123225


In [16]:
df.shape

(7069, 3)